In [1]:
import util
from journal_api.journal_api import JournalAPI
from IPython.display import JSON
import xml.etree.ElementTree as ET
import json
import numpy as np
import pandas as pd
import logging
import logging.config
import yaml
from journal_api import semanticscholar_api

2023-08-29 18:29:27,146 - INFO - NumExpr defaulting to 8 threads.
2023-08-29 18:29:32,835 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-29 18:29:33,111 - DEBUG - Creating converter from 7 to 5
2023-08-29 18:29:33,112 - DEBUG - Creating converter from 5 to 7
2023-08-29 18:29:33,114 - DEBUG - Creating converter from 7 to 5
2023-08-29 18:29:33,116 - DEBUG - Creating converter from 5 to 7


In [2]:
with open('logging_config.yaml', 'r') as config_file:
    log_config = yaml.safe_load(config_file)
logging.config.dictConfig(log_config)

In [3]:
title = "Design and Control of Roller Grasper V2 for In-Hand Manipulation"

In [4]:
journal = JournalAPI()
dt = journal.search(title, 2018, find_references=False, fix_references=False)
JSON(dt)

2023-08-29 18:29:36,836 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation


<IPython.core.display.JSON object>

In [5]:
def source_target(source, target):
    ret = pd.DataFrame(columns=["source", "target"])
    for t in target:
        ret = ret.append({"source": source, "target": t}, ignore_index=True)
    return ret

In [6]:
df = None
def source_target_generator(title, title_list, level = 1, start_year = 2017, fix_references = False):
    global df
    ret = pd.DataFrame(columns=["source", "target"])
    if(level > 0):
        logging.info(f"source_target_generator: level={level}, len={len(title_list)}, title={title}")
        for index, title in enumerate(title_list):
            logging.info(f"source_target_generator: ({index+1}/{len(title_list)}) title={title}")
            src = journal.search(title, start_year, find_references=True, fix_references=fix_references)
            if src != None:
                if df is None or df.empty:
                    df = pd.DataFrame([src])
                else:
                    matching_rows = df[(df["title"] == src["title"])]
                    if matching_rows.empty:
                        new_row = pd.Series(src)
                        df = df.append(new_row, ignore_index=True)
                    else:
                        logging.debug(f"source_target_generator: Title already exists. title={src['title']}")
                
                st = source_target(src["title"],src["references"])
                ret = pd.concat([ret, st], ignore_index=True)
                lv = level - 1
                if lv == 1:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year)], ignore_index=True)
                else:
                    ret = pd.concat([ret, source_target_generator(src["title"], src["references"], lv, start_year, fix_references)], ignore_index=True)
            
    return ret

In [7]:
x = source_target_generator("", [title], 2, 2018, False)

2023-08-29 18:30:11,968 - INFO - source_target_generator: level=2, len=1, title=
2023-08-29 18:30:11,970 - INFO - source_target_generator: (1/1) title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-29 18:30:12,574 - DEBUG - ss.search: Found! title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-29 18:30:12,594 - INFO - source_target_generator: level=1, len=8, title=Design and Control of Roller Grasper V2 for In-Hand Manipulation
2023-08-29 18:30:12,595 - INFO - source_target_generator: (1/8) title=Concept2Robot: Learning manipulation concepts from instructions and human demonstrations
2023-08-29 18:30:13,215 - DEBUG - ss.search: Found! title=Concept2Robot: Learning manipulation concepts from instructions and human demonstrations
2023-08-29 18:30:13,288 - INFO - source_target_generator: (2/8) title=Design of a Roller-Based Dexterous Hand for Object Grasping and Within-Hand Manipulation
2023-08-29 18:30:13,986 - DEBUG - ss.search: Found!

In [8]:
df.head()

,paperId,title,abstract,year,citationCount,influentialCitationCount,fieldsOfStudy,tldr,authors,references,source
0,5d0c16d8532b4514dc4a330562e44d21c3dbaeff,Design and Control of Roller Grasper V2 for In...,The ability to perform in-hand manipulation st...,2020,25,0,"[Computer Science, Engineering]",This work presents a novel non-anthropomorphic...,"[Shenli Yuan, Lin Shao, Connor L. Yako, Alexan...",[Concept2Robot: Learning manipulation concepts...,semanticscholar
1,c328a1ea31eaa4b357e584285ffb6ff2671b5bd4,Concept2Robot: Learning manipulation concepts ...,We aim to endow a robot with the ability to le...,2020,92,5,[Computer Science],This work aims to endow a robot with the abili...,"[Lin Shao, Toki Migimatsu, Qiang Zhang, Karen ...",[Model-Based Inverse Reinforcement Learning fr...,semanticscholar
2,b019519f00715061b4fa05c7e8eda768f3b4779d,Design of a Roller-Based Dexterous Hand for Ob...,This paper describes the development of a nove...,2020,33,1,[Computer Science],This paper describes the development of a nove...,"[Shenli Yuan, A. D. Epps, Jerome B. Nowak, J. ...",[Design\nand Analysis of a Multimodal Grasper ...,semanticscholar
3,e5a3aca5321fc0e1b70fb51e4c4b6ff9dfbdb05d,Design\nand Analysis of a Multimodal Grasper H...,"\n This paper presents the design, analysis, a...",2019,13,0,[Computer Science],"This paper presents the design, analysis, and ...","[Nagamanikandan Govindan, Asokan Thondiyath]",[Variable-Friction Finger Surfaces to Enable W...,semanticscholar
4,f12e6e06816b1ff34bb0673a9059065458c43bc5,A Century of Robotic Hands,This article reports on the state of the art o...,2019,166,7,"[Computer Science, Engineering]",,"[C. Piazza, G. Grioli, M. G. Catalano, A. Bicchi]",[The evolution of functional hand replacement:...,semanticscholar


In [9]:
x

,source,target
0,Design and Control of Roller Grasper V2 for In...,Concept2Robot: Learning manipulation concepts ...
1,Design and Control of Roller Grasper V2 for In...,Design of a Roller-Based Dexterous Hand for Ob...
2,Design and Control of Roller Grasper V2 for In...,Design\nand Analysis of a Multimodal Grasper H...
3,Design and Control of Roller Grasper V2 for In...,A Century of Robotic Hands
4,Design and Control of Roller Grasper V2 for In...,Dual-Arm In-Hand Manipulation and Regrasping U...
...,...,...
257,Learning dexterous in-hand manipulation,Distributed Distributional Deterministic Polic...
258,Learning dexterous in-hand manipulation,On Policy Learning Robust to Irreversible Even...
259,Learning dexterous in-hand manipulation,Rectiﬁed Linear Units Improve Restricted Boltz...
260,In-Hand Manipulation via Motion Cones,"Pushing revisited: Differential flatness, traj..."


In [11]:
for t in x["target"]:
    if len(df[df["title"] == t]) == 0:
        r = journal.search(t, 2018, find_references=False, fix_references=False)
        if r != None:
            new_row = pd.Series(r)
            df = df.append(new_row, ignore_index=True)

2023-08-29 18:37:49,370 - DEBUG - ss.search: Found! title=Model-Based Inverse Reinforcement Learning from Visual Demonstrations
2023-08-29 18:37:49,371 - WARNING - ss._remake_references: Unknown year. title=Let's do inverse rl
2023-08-29 18:37:49,372 - WARNING - ss._remake_references: Unknown year. title=The parameters of f dyn are trained by optimizing a normalized mean squared error (NMSE) between predictedŝ t+1 = f dyn (s t , u t ) and the ground truth s t+1
2023-08-29 18:37:49,374 - WARNING - ss._remake_references: Unknown year. title=We change their inner loop, to use our model-based trajectory optimisation routine. Further, we saw fair to use as features φ (·) the per-step task objective L IRL (·) employed in this work
2023-08-29 18:37:49,959 - DEBUG - ss.search: Found! title=GRAC: Self-Guided and Self-Regularized Actor-Critic
2023-08-29 18:37:50,706 - DEBUG - ss.search: Found! title=Robot Object Retrieval with Contextual Natural Language Queries
2023-08-29 18:37:51,387 - DEBUG -

In [12]:
df.to_json("output_1.json", orient="records", indent=2)
x.to_csv("output_1.csv", index=False)

In [ ]:
import json

# Open the JSON file for reading
with open("output.json", "r") as json_file:
    data = json.load(json_file)

journal = JournalAPI()
for d in data["nodes"]:
    dt = journal.search(d["name"], 2018, find_references=False, fix_references=False)
    print(dt)

In [ ]:
from collections import defaultdict
from pyvis.network import Network

net = Network("1200px", "1200px", notebook=True)

# Example dummy data
sources = x["source"]
targets = x["target"]
sources_short = [source[:100] + "..." for source in sources]
targets_short = [target[:100] + "..." for target in targets]

# Create a dictionary to store the count of each target
target_counts = defaultdict(int)
for t in targets_short:
    target_counts[t] += 10
    if t == "...":
        target_counts[t] = 1

# Create unique node labels for each occurrence of a node
unique_nodes = {}
for node in set(sources_short + targets_short):
    count = target_counts[node]
    size = count if count > 1 else 1  # Ensuring the smallest size is 1
    unique_label = f"{node} ({count})"
    net.add_node(unique_label, size=size)
    unique_nodes[node] = unique_label

# Add edges to the graph using the unique node labels
for s, t in zip(sources_short, targets_short):
    net.add_edge(unique_nodes[s], unique_nodes[t])

# Step 3: Visualize the graph in Jupyter Notebook
net.show("network_graph2.html")
